### <font color="#1F618D"> Newyork City Job Postings - Data Engineering Challenge </font>
### <font color="#F5B041"> Part-1: Data Profiling </font>

In [15]:
# Import the findspark module to help locate and initialize Spark
import findspark

# Initialize Spark
findspark.init()

# Import the jupyter_black module, which is an extension to format code cells in Jupyter Notebook
import jupyter_black

# Load and enable the jupyter_black extension to format code cells automatically
jupyter_black.load()

In [16]:
# importing required libraries

import pandas as pd

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, count, isnan, when

from common import *

In [17]:
def set_spark_conf():
    spark_conf = SparkConf()

    # Set the number of executor cores
    # spark_conf.set("spark.executor.cores", "2")

    # Set the executor memory
    # spark_conf.set("spark.executor.memory", "1g")

    # Set the driver memory
    # spark_conf.set("spark.driver.memory", "2g")

    # Additional settings
    spark_conf.set("spark.master", "spark://master:7077")

    return spark_conf


def get_spark_session(spark_conf: dict, job_name: str) -> SparkSession:
    """
    Get or create a SparkSession.

    Parameters:
        spark_conf (dict): Configuration options for SparkSession.
        job_name (str): Name of the Spark job.

    Returns:
        SparkSession: The initialized or retrieved SparkSession.
    """
    spark = SparkSession.builder.appName(job_name).config(conf=spark_conf).getOrCreate()
    return spark

In [18]:
# set the spark configuration options
spark_conf = set_spark_conf()

# Create a SparkSession with the configured settings
spark = get_spark_session(spark_conf, "NYCJobsDataProfiling")

In [ ]:
spark.catalog.listTables()

In [5]:
# load the input dataset

df = spark.read.csv(
    "../../dataset/raw_data/nyc-jobs.csv", header=True, inferSchema=True, escape='"'
)

In [6]:
# obtain the column information

df.printSchema()

root
 |-- Job ID: integer (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Posting Type: string (nullable = true)
 |-- # Of Positions: integer (nullable = true)
 |-- Business Title: string (nullable = true)
 |-- Civil Service Title: string (nullable = true)
 |-- Title Code No: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- Full-Time/Part-Time indicator: string (nullable = true)
 |-- Salary Range From: double (nullable = true)
 |-- Salary Range To: double (nullable = true)
 |-- Salary Frequency: string (nullable = true)
 |-- Work Location: string (nullable = true)
 |-- Division/Work Unit: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Minimum Qual Requirements: string (nullable = true)
 |-- Preferred Skills: string (nullable = true)
 |-- Additional Information: string (nullable = true)
 |-- To Apply: string (nullable = true)
 |-- Hours/Shift: string (nullable = true)
 |-- Work Locat

### <font color="#1F618D"> Detailed analysis of source data: <font color="#1F618D">

The provided schema represents the structure of a PySpark DataFrame that holds information about job postings in New York City. 
Each row in the DataFrame corresponds to a single job posting, and the columns represent various attributes associated with 
each posting. Let us analyze the source data based on the provided schema, categorizing columns by their data types.

#### <span style="text-decoration: underline;"> Numerical Columns:</span>

**#Of Positions (numerical):** This column represents the number of positions available for the job. It contains numerical values.

**Salary Range From (numerical):** This column represents the starting point of the salary range for the job. It contains numerical values.

**Salary Range To (numerical):** This column represents the endpoint of the salary range for the job. It contains numerical values.

#### <span style="text-decoration: underline;"> Categorical Columns:</span> 

**Job ID (categorical):** This column is a unique identifier for each job posting. It contains categorical values.

**Agency (categorical):** This column represents the agency or organization offering the job. It contains categorical values.

**Posting Type (categorical):** This column indicates the type of job posting. It contains categorical values.

**Business Title (categorical):** This column contains the business titles of the jobs. It contains categorical values.

**Civil Service Title (categorical):** This column contains the civil service titles of the jobs. It contains categorical values.

**Title Code No (categorical):** This column contains unique code numbers for job titles. It contains categorical values.

**Level (categorical):** This column represents the level or rank associated with the job. It contains categorical values.

**Job Category (categorical):** This column represents the category or field to which the job belongs. It contains categorical values.

**Full-Time/Part-Time indicator (categorical):** This column indicates whether the job is full-time or part-time. It contains categorical values.

**Salary Frequency (categorical):** This column represents the frequency at which the salary is paid. It contains categorical values.

**Work Location (categorical):** This column represents the primary location where the job will be performed. It contains categorical values.

**Division/Work Unit (categorical):** This column represents the division or specific unit within the agency where the job belongs. It contains categorical values.

**Residency Requirement (categorical):** This column specifies any residency requirements for the job. It contains categorical values.

**Posting Date (categorical):** This column represents the date when the job posting was initially published. It contains categorical date values.

**Post Until (categorical):** This column represents the date until which the job posting will be available. It contains categorical date values.

**Posting Updated (categorical):** This column represents the date when the job posting information was last updated. It contains categorical date values.

**Process Date (categorical):** This column represents the date when the information was processed. It contains categorical date values.

#### <span style="text-decoration: underline;"> Textual Columns:</span> 

**Job Description (textual):** This column contains descriptions of the responsibilities and tasks associated with the job.

**Minimum Qual Requirements (textual):** This column contains the minimum qualifications or requirements that candidates must meet to be eligible for the job.

**Preferred Skills (textual):** This column contains skills or qualifications that are preferred but not mandatory for the job.

**Additional Information (textual):** This column contains additional information about the job posting.

**To Apply (textual):** This column contains instructions on how to apply for the job.

**Hours/Shift (textual):** This column contains information about the working hours or shifts associated with the job.

**Work Location 1 (textual):** This column contains additional information about the work location.

**Recruitment Contact (textual):** This column contains contact details for inquiries related to recruitment. From the data profiling, noticed that it contains all null values, hence can be removed.

This detailed analysis categorizes the columns in the provided schema into numerical, categorical, and textual types, providing insights into the nature of the data present in each column.

### <font color="#1F618D"> Data Profiling </font>

In [7]:
def get_data_types(data_df: DataFrame, columns: list) -> pd.DataFrame:
    """
    Get data types of specified columns in the DataFrame.

    Parameters:
        data_df (DataFrame): The input DataFrame.
        columns (list): List of column names.

    Returns:
        pd.DataFrame: DataFrame containing column names and their data types.
    """
    data_types_df = pd.DataFrame(
        {
            "column_names": columns,
            "data_types": [x[1] for x in data_df.select(columns).dtypes],
        }
    )
    return data_types_df[["column_names", "data_types"]]


def get_null_counts(data_df: DataFrame, columns: list) -> pd.DataFrame:
    """
    Get counts of null and NaN values for specified columns in the DataFrame.

    Parameters:
        data_df (DataFrame): The input DataFrame.
        columns (list): List of column names.

    Returns:
        pd.DataFrame: DataFrame containing column names and their null value counts.
    """
    null_counts_df = (
        data_df.select(
            [
                count(when(isnan(c) | col(c).isNull(), c)).alias(c)
                for c in columns
                if data_df.select(c).dtypes[0][1] != "timestamp"
            ]
        )
        .toPandas()
        .transpose()
    )
    null_counts_df = null_counts_df.reset_index()
    null_counts_df.columns = ["column_names", "num_null"]
    return null_counts_df


def get_space_and_blank_counts(data_df: DataFrame, columns: list) -> pd.DataFrame:
    """
    Get counts of white spaces and blanks for specified columns in the DataFrame.

    Parameters:
        data_df (DataFrame): The input DataFrame.
        columns (list): List of column names.

    Returns:
        pd.DataFrame: DataFrame containing column names and their space/blank counts.
    """
    num_spaces = [data_df.where(col(c).rlike("^\\s+$")).count() for c in columns]
    num_blank = [data_df.where(col(c) == "").count() for c in columns]

    space_blank_df = pd.DataFrame(
        {"column_names": columns, "num_spaces": num_spaces, "num_blank": num_blank}
    )
    return space_blank_df


def get_descriptive_stats(data_df: DataFrame) -> pd.DataFrame:
    """
    Get descriptive statistics for specified columns in the DataFrame.

    Parameters:
        data_df (DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing descriptive statistics.
    """
    desc_df = data_df.describe().toPandas().transpose()
    desc_df.columns = ["count", "mean", "stddev", "min", "max"]
    desc_df = desc_df.iloc[1:, :]
    desc_df = desc_df.reset_index()
    desc_df.columns.values[0] = "column_names"
    desc_df = desc_df[["column_names", "count", "mean", "stddev"]]
    return desc_df


def get_distinct_value_counts(data_df: DataFrame, columns: list) -> pd.DataFrame:
    """
    Get the number of distinct values for specified columns in the DataFrame.

    Parameters:
        data_df (DataFrame): The input DataFrame.
        columns (list): List of column names.

    Returns:
        pd.DataFrame: DataFrame containing column names and their distinct value counts.
    """
    distinct_counts_df = pd.DataFrame(
        {
            "column_names": columns,
            "num_distinct": [data_df.select(x).distinct().count() for x in columns],
        }
    )
    return distinct_counts_df


def get_most_frequent_values(data_df: DataFrame, columns: list) -> pd.DataFrame:
    """
    Get the most frequently occurring value and its count for specified columns in the DataFrame.

    Parameters:
        data_df (DataFrame): The input DataFrame.
        columns (list): List of column names.

    Returns:
        pd.DataFrame: DataFrame containing column names, most frequent value, and its count.
    """
    most_freq_values = [
        data_df.groupBy(x)
        .count()
        .sort("count", ascending=False)
        .limit(1)
        .toPandas()
        .iloc[0]
        .tolist()
        for x in columns
    ]
    most_freq_values_df = pd.DataFrame(
        most_freq_values, columns=["most_freq_value", "most_freq_value_count"]
    )
    most_freq_values_df["column_names"] = columns
    most_freq_values_df = most_freq_values_df[
        ["column_names", "most_freq_value", "most_freq_value_count"]
    ]
    return most_freq_values_df


def get_least_frequent_values(data_df: DataFrame, columns: list) -> pd.DataFrame:
    """
    Get the least frequently occurring value and its count for specified columns in the DataFrame.

    Parameters:
        data_df (DataFrame): The input DataFrame.
        columns (list): List of column names.

    Returns:
        pd.DataFrame: DataFrame containing column names, least frequent value, and its count.
    """
    least_freq_values = [
        data_df.groupBy(x)
        .count()
        .sort("count", ascending=True)
        .limit(1)
        .toPandas()
        .iloc[0]
        .tolist()
        for x in columns
    ]
    least_freq_values_df = pd.DataFrame(
        least_freq_values, columns=["least_freq_value", "least_freq_value_count"]
    )
    least_freq_values_df["column_names"] = columns
    least_freq_values_df = least_freq_values_df[
        ["column_names", "least_freq_value", "least_freq_value_count"]
    ]
    return least_freq_values_df


def is_categorical_column(column: DataFrame, distinct_threshold: int) -> bool:
    """
    Determine if a column is categorical based on the distinct value threshold.

    Parameters:
        column (DataFrame): The input column to be evaluated.
        distinct_threshold (int): The threshold to consider a column as categorical.

    Returns:
        bool: True if the column is categorical, False otherwise.
    """
    num_distinct = column.nunique()
    return num_distinct <= distinct_threshold


def profile_column_values(data_df: DataFrame, column_name: str):
    """
    Profile the values of a column by showing value counts.

    Parameters:
        data_df (DataFrame): The input DataFrame.
        column_name (str): The name of the column to be profiled.
    """
    value_counts = data_df.groupBy(column_name).count().orderBy(col("count").desc())

    return value_counts.toPandas()

In [11]:
def get_data_profile(data_all_df: DataFrame, columns2Bprofiled: list) -> pd.DataFrame:
    """
    Generate a comprehensive data profile for specified columns in the DataFrame.

    Parameters:
        data_all_df (DataFrame): The input DataFrame.
        columns2Bprofiled (list): List of column names to be profiled.

    Returns:
        pd.DataFrame: Comprehensive data profiling results.
    """
    # Print the section heading
    print_section_heading("DATA PROFILING REPORT")

    # --- Data Shape ---
    print_subheading("Data Shape")

    # Get the number of rows and columns
    num_rows = data_all_df.count()
    num_columns = len(data_all_df.columns)

    # Display the results
    print(f"Number of rows: {num_rows}")
    print(f"Number of columns: {num_columns}")

    # Dropping duplicates
    data_all_df = data_all_df.distinct()
    dist_tot_cnt = data_all_df.count()
    print(f"Total Records after dropping duplicates: {dist_tot_cnt}")

    print_subheading_closure()

    # --- Column Stats ---
    print_subheading("Column Stats")

    # Select columns to be profiled
    data_df = data_all_df.select(columns2Bprofiled)

    # Get data type and other basic statistics for columns
    prof_df = get_data_types(data_df, columns2Bprofiled)

    # Count the number of rows
    num_rows = data_df.count()
    prof_df["num_rows"] = num_rows

    # Get null counts for columns
    null_counts_df = get_null_counts(data_df, columns2Bprofiled)
    prof_df = pd.merge(prof_df, null_counts_df, on=["column_names"], how="left")

    # Get counts of spaces and blank values for columns
    space_blank_df = get_space_and_blank_counts(data_df, columns2Bprofiled)
    prof_df = pd.merge(prof_df, space_blank_df, on=["column_names"], how="left")

    # Get descriptive statistics for columns
    desc_df = get_descriptive_stats(data_df)
    prof_df = pd.merge(prof_df, desc_df, on=["column_names"], how="left")

    # Get distinct value counts for columns
    distinct_counts_df = get_distinct_value_counts(data_df, columns2Bprofiled)
    prof_df = pd.merge(prof_df, distinct_counts_df, on=["column_names"], how="left")

    # Get most frequent values for columns
    most_freq_values_df = get_most_frequent_values(data_df, columns2Bprofiled)
    prof_df = pd.merge(prof_df, most_freq_values_df, on=["column_names"], how="left")

    # Get least frequent values for columns
    least_freq_values_df = get_least_frequent_values(data_df, columns2Bprofiled)
    prof_df = pd.merge(prof_df, least_freq_values_df, on=["column_names"], how="left")

    # Display the comprehensive data profile DataFrame
    display(prof_df)

    print_subheading_closure()

    # --- Column Profiles ---
    print_subheading("Column Frequency Distribution")

    print("Getting value counts - top 10")
    for col_name in columns2Bprofiled:
        display(profile_column_values(data_df, col_name).head(10))

    print_subheading_closure()

In [12]:
get_data_profile(df, df.columns)


╔════════════════════════════════════════════════════════╗
║                 DATA PROFILING REPORT                  ║
╚════════════════════════════════════════════════════════╝

╔════════════════════════════════════════════════════════╗
║                       Data Shape                       ║
╠════════════════════════════════════════════════════════╣
Number of rows: 2946
Number of columns: 28
Total Records after dropping duplicates: 2915
╚════════════════════════════════════════════════════════╝

╔════════════════════════════════════════════════════════╗
║                      Column Stats                      ║
╠════════════════════════════════════════════════════════╣


,column_names,data_types,num_rows,num_null,num_spaces,num_blank,count,mean,stddev,num_distinct,most_freq_value,most_freq_value_count,least_freq_value,least_freq_value_count
0,Job ID,int,2915,0.0,0,0,2915,384863.0401372213,53017.480279010946,1661,384143,3.0,363130,1.0
1,Agency,string,2915,0.0,0,0,2915,None,None,52,DEPT OF ENVIRONMENT PROTECTION,650.0,PUBLIC ADMINISTRATOR-NEW YORK,1.0
2,Posting Type,string,2915,0.0,0,0,2915,None,None,2,Internal,1653.0,External,1262.0
3,# Of Positions,int,2915,0.0,0,0,2915,2.4363636363636365,8.58105496276755,34,1,2263.0,120,1.0
4,Business Title,string,2915,0.0,0,0,2915,None,None,1244,Assistant Civil Engineer,32.0,Administrative Program Coordinator,1.0
5,Civil Service Title,string,2915,0.0,0,0,2915,None,None,312,COMMUNITY COORDINATOR,180.0,ADM HOUSING DEV SPEC(NON MGRL),1.0
6,Title Code No,string,2915,0.0,0,0,2915,35536.76237989653,28113.970668785427,323,56058,180.0,31118,1.0
7,Level,string,2915,0.0,0,0,2915,1.0541530944625408,1.1402285900872402,14,0,1097.0,4B,1.0
8,Job Category,string,2915,2.0,0,0,2913,None,None,131,"Engineering, Architecture, & Planning",497.0,"Administration & Human Resources Technology, D...",1.0
9,Full-Time/Part-Time indicator,string,2915,193.0,0,0,2722,None,None,3,F,2597.0,P,125.0


╚════════════════════════════════════════════════════════╝

╔════════════════════════════════════════════════════════╗
║             Column Frequency Distribution              ║
╠════════════════════════════════════════════════════════╣
Getting value counts - top 10


,Job ID,count
0,384143,3
1,425375,2
2,412113,2
3,396195,2
4,369120,2
5,383183,2
6,243640,2
7,422886,2
8,274845,2
9,416166,2


,Agency,count
0,DEPT OF ENVIRONMENT PROTECTION,650
1,NYC HOUSING AUTHORITY,231
2,DEPT OF HEALTH/MENTAL HYGIENE,187
3,DEPARTMENT OF TRANSPORTATION,180
4,DEPT OF DESIGN & CONSTRUCTION,142
5,TAXI & LIMOUSINE COMMISSION,134
6,ADMIN FOR CHILDREN'S SVCS,108
7,DEPT OF INFO TECH & TELECOMM,107
8,LAW DEPARTMENT,90
9,HOUSING PRESERVATION & DVLPMNT,86


...

,Posting Date,count
0,2019-11-01,64
1,2019-09-30,59
2,2019-11-26,58
3,2019-11-27,54
4,2019-12-10,53
5,2019-12-13,51
6,2019-12-06,50
7,2019-12-11,48
8,2019-10-28,45
9,2019-10-31,38


,Post Until,count
0,NaT,2052
1,2019-12-20,51
2,2019-12-22,38
3,2019-12-18,34
4,2020-02-24,32
5,2020-01-26,32
6,2020-02-25,31
7,2019-12-26,31
8,2019-12-21,31
9,2019-12-19,28


,Posting Updated,count
0,2019-12-13,82
1,2019-12-16,71
2,2019-12-11,68
3,2019-11-27,67
4,2019-12-10,66
5,2019-11-26,66
6,2019-12-12,61
7,2019-12-05,52
8,2019-12-06,47
9,2019-11-04,45


,Process Date,count
0,2019-12-17,2911
1,NaT,4


╚════════════════════════════════════════════════════════╝


In [13]:
spark.stop()